In [34]:
import requests
from datetime import datetime
from urllib import parse
import pandas as pd
import json
import os
import numpy as np

In [35]:
URL = "https://wikimedia.org/api/rest_v1/"
HEADERS = {"Accept":"application/json","user-agent":"test123@gmail.com"}
SESSION = requests.Session()

In [36]:
def get_pageviews(article: str, start: datetime, end: datetime, project="en.wikipedia.org",
                  access="all-access", agent="all-agents", granularity="daily"):
    """
        args:
            article: The name of the article
            start: Start date 
            end: End date
            project: The domain, default en.wikipedia.org
            access: Type of the device, default all-access. other options(Desktop, mobile-app, mobile-web)
            agent: Type of the agent, default all-agents. other options(user, spider, automated)
            granularity: The time unit, default daily. Other options(monthly)
            
    """
    params = [
        "metrics",
        "pageviews",
        "per-article",
        project.capitalize(),
        access,
        agent,
        parse.quote(article),
        granularity,
        start.strftime("%Y%m%d"),
        end.strftime("%Y%m%d")
    ]
    url = URL + "/".join(params)
    return SESSION.get(url,headers=HEADERS).json()["items"]


In [37]:
def store_pageviews(title, genre: str, data):
    """
        args:
            genre: options (Games,Film and TV, Literary, Music)
    """
    if genre.lower() not in ["games", "film and tv", "literary", "music"]:
        raise ValueError(
            "Genre must be an element of [Games, Film and TV, Literary, Music]")
    df = pd.DataFrame(data)
    df["timestamp"] = pd.to_datetime(df["timestamp"], format="%Y%m%d%H",)
    if not os.path.isdir("Data"):
        os.mkdir("Data")
    df.to_excel("Data/" + genre +
                "/" + title + ".xlsx", index=False)


def fetch_data_sources(page, filename):
    url = "https://www.wikitable2json.com/api/" + \
        page

    params = {"cleanRef": "true", "lang": "en"}
    request = SESSION.get(url=url, params=params, headers=HEADERS)
    resposne = request.json()
    table = np.squeeze(resposne)
    data = {}
    for i in range(len(table[0])):
      data[table[i, i]] = table[1:, i].tolist()
    df = pd.DataFrame(data)
    df.to_excel("Data/" + filename + ".xlsx", index=False)


In [38]:
fetch_data_sources("List_of_video_games_considered_the_best", "Games_sources")